# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

from dotenv import load_dotenv
load_dotenv()

True

In [12]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [9]:
import os
from glob import glob


# Get PRICE_DATA directory from environment variables
price_data_dir = os.environ.get("PRICE_DATA")

# Recursively find all parquet files within the PRICE_DATA folder
parquet_files = glob(os.path.join(price_data_dir, "**/*.parquet"), recursive=True)

# Preview results
print(f"Found {len(parquet_files)} parquet files.")
parquet_files[:3]


Found 3057 parquet files.


['../../05_src/data/prices\\ACN\\ACN_2001\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2001\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.0.parquet']

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [16]:
# Read all parquet files
ddf = dd.read_parquet(parquet_files, engine="pyarrow")

# Use map_partitions to do lag safely
def add_features(df):
    df = df.sort_values("Date")
    df["Close_lag_1"] = df["Close"].shift(1)
    df["Adj_Close_lag_1"] = df["Adj Close"].shift(1)
    df["returns"] = (df["Close"] / df["Close_lag_1"]) - 1
    df["hi_lo_range"] = df["High"] - df["Low"]
    return df

# Apply per ticker using groupby + map_partitions
meta = {
    'Date': 'datetime64[ns]',
    'Open': 'float64',
    'High': 'float64',
    'Low': 'float64',
    'Close': 'float64',
    'Adj Close': 'float64',
    'Volume': 'int64',
    'source': 'object',
    'ticker': 'object',
    'Year': 'int64',  
    'Close_lag_1': 'float64',
    'Adj_Close_lag_1': 'float64',
    'returns': 'float64',
    'hi_lo_range': 'float64'
}
dd_feat = ddf.groupby("ticker").apply(add_features, meta=meta)
# Preview results
dd_feat.head()


,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [19]:
# Convert a subset to Pandas for testing
dd_sample = dd_feat[dd_feat["ticker"] == "AAPL"].compute()

# Sort by date to ensure rolling is correct
dd_sample = dd_sample.sort_values("Date")

# Compute the 10-day moving average of returns
dd_sample["returns_ma_10"] = dd_sample["returns"].rolling(10).mean()

# Preview
dd_sample[["Date", "ticker", "returns", "returns_ma_10"]].head(15)


,Date,ticker,returns,returns_ma_10


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

### Reflection

It was not strictly necessary to convert to Pandas to calculate the moving average of returns. Dask also supports `.rolling()` operations, but it requires additional configuration (like setting the index properly and ensuring correct partitioning).

However, converting to Pandas made the computation easier and more intuitive for this small subset. If the dataset were much larger, staying in Dask would be preferred to avoid memory issues and take advantage of parallelism.

In summary:
- For small subsets: Pandas is simpler and faster.
- For full datasets: Using Dask with `rolling()` is more scalable, though slightly more complex to configure.


## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [X] Created a branch with the correct naming convention.
- [X] Ensured that the repository is public.
- [X] Reviewed the PR description guidelines and adhered to them.
- [X] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.